<a href="https://colab.research.google.com/github/MoreCookies/fastai-scifair24/blob/main/FastAI_scifair24_Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import urllib.request
import zipfile


urls = {
    "train" : "https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task3_Training_Input.zip",
    "train_ground_truth" : "https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task3_Training_GroundTruth.zip",
    "validation" : "https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task3_Validation_Input.zip",
    "validation_ground_truth" : "https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task3_Validation_GroundTruth.zip",
    "test" : "https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task3_Test_Input.zip",
    "test_ground_truth" : "https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task3_Test_GroundTruth.zip"
    #"train_seg": "https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task1-2_Training_Input.zip",
    #"train_seg_gt": "https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task1_Training_GroundTruth.zip",
    #"validation_seg": "https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task1-2_Validation_Input.zip",
    #"validation_seg_gt": "https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task1_Validation_GroundTruth.zip",
    #"train_2019" : "https://isic-challenge-data.s3.amazonaws.com/2019/ISIC_2019_Training_Input.zip",
    #"train_gt_2019" : "https://isic-challenge-data.s3.amazonaws.com/2019/ISIC_2019_Training_GroundTruth.csv"

}

def download_all():
    for item in urls.items():
        name = item[0]
        url = item[1]
        filename = f"{name}.csv"
        if not url.endswith(".csv"):
          filename = f"{name}.zip"
        if os.path.exists(filename):
            print(f"already exists file {filename}")
        else:
            print(f"downloading {filename} from {url} ...")
            urllib.request.urlretrieve(url, filename)
    print(f"done downloading all")

def unzip_all():
    for item in urls.items():
        name = item[0]
        url = item[1]
        filename = f"{name}.zip"
        print(filename)

        with zipfile.ZipFile(filename, 'r') as zip_ref:
            print(f"unzipping {filename} to folder")
            zip_ref.extractall(f"{name}")
    print(f"done unzipping all")


if __name__ == "__main__":
    download_all()
    unzip_all()

already exists file train.zip
already exists file train_ground_truth.zip
already exists file validation.zip
already exists file validation_ground_truth.zip
already exists file test.zip
already exists file test_ground_truth.zip
done downloading all
train.zip
unzipping train.zip to folder
train_ground_truth.zip
unzipping train_ground_truth.zip to folder
validation.zip
unzipping validation.zip to folder
validation_ground_truth.zip
unzipping validation_ground_truth.zip to folder
test.zip
unzipping test.zip to folder
test_ground_truth.zip
unzipping test_ground_truth.zip to folder
done unzipping all


In [ ]:
from fastai.vision.all import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import shutil
import os

from glob import glob

In [ ]:
shutil.copy("/content/train_ground_truth/ISIC2018_Task3_Training_GroundTruth/ISIC2018_Task3_Training_GroundTruth.csv", "/content/train/ISIC2018_Task3_Training_Input")

'/content/train/ISIC2018_Task3_Training_Input/ISIC2018_Task3_Training_GroundTruth.csv'

In [ ]:
def format_csv(df, target=0):
  #add extension
  for i in range(len(df)):
    if ".jpg" not in df["image"].iloc[i]:
      df.at[i, 'image'] = df["image"].iloc[i] + ".jpg"

  #create dx column
  lesion_cols = df.drop("image", axis=1).columns
  if 'dx' in df.columns:
    lesion_cols = df.drop(["image", "dx"], axis=1).columns

  df["dx"] = df[lesion_cols].idxmax(axis=1)
  df = df.fillna(0.0)

  if target != 0:
    df.to_csv(target, index_label=False)
  return df

In [ ]:
path = "/content/train/ISIC2018_Task3_Training_Input/ISIC2018_Task3_Training_GroundTruth.csv"

gt = pd.read_csv(path)

In [ ]:
gt2 = format_csv(gt)

Index(['MEL', 'NV', 'BCC', 'AKIEC', 'BKL', 'DF', 'VASC'], dtype='object')


In [ ]:
gt2.sample(10)

,image,MEL,NV,BCC,AKIEC,BKL,DF,VASC,dx
7907,ISIC_0032213.jpg,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NV
9149,ISIC_0033455.jpg,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NV
6316,ISIC_0030622.jpg,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NV
4341,ISIC_0028647.jpg,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NV
3928,ISIC_0028234.jpg,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NV
2832,ISIC_0027138.jpg,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NV
7356,ISIC_0031662.jpg,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NV
6689,ISIC_0030995.jpg,1.0,0.0,0.0,0.0,0.0,0.0,0.0,MEL
7495,ISIC_0031801.jpg,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NV
5620,ISIC_0029926.jpg,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NV


In [ ]:
def get_intersection(df_1, df_2):
  df1 = set(df_1["image"])
  df2 = set(df_2["image"])

  intersection = df1.intersection(df2)
  AnotB = df1.difference(df2)
  BnotA = df2.difference(df1)

  return intersection, AnotB, BnotA

In [ ]:
def merge_data(target_path, target_csv, img_paths, csv_paths, target):
  targ_csv = pd.read_csv(target_csv)

  for i in range(len(img_paths)):
    gt = pd.read_csv(csv_paths[i])
    inter = get_intersection(targ_csv, gt)
    #move images
    for x in inter[2]:
      if not x.endswith(".jpg"):
        shutil.copy(os.path.join(img_paths[i], x+".jpg"), os.path.join(target_path, x+".jpg"))
      else:
        shutil.copy(os.path.join(img_paths[i], x+".jpg"), os.path.join(target_path, x+".jpg"))

    #merge datasets
    targ_csv = pd.concat([targ_csv, gt]).drop_duplicates().reset_index(drop=True)

  print("done merge")
  return format_csv(targ_csv, target)


In [ ]:
target_path = "/content/train/ISIC2018_Task3_Training_Input"
target_csv = "/content/train/ISIC2018_Task3_Training_Input/ISIC2018_Task3_Training_GroundTruth.csv"

img_paths = ["/content/validation/ISIC2018_Task3_Validation_Input", "/content/train_2019/ISIC_2019_Training_Input"]
csv_paths = ["/content/validation_ground_truth/ISIC2018_Task3_Validation_GroundTruth/ISIC2018_Task3_Validation_GroundTruth.csv", "/content/train_gt_2019.csv"]

target = "/content/train/ISIC2018_Task3_Training_Input/Training_GroundTruth_Merged.csv"

gt = merge_data(target_path, target_csv, img_paths, csv_paths, target)

done merge


In [ ]:
gt2 = gt.fillna(0.0)

In [ ]:
gt2.sample(10)

,image,MEL,NV,BCC,AKIEC,BKL,DF,VASC,AK,SCC,UNK,dx
7908,ISIC_0032214.jpg,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,MEL
8252,ISIC_0032558.jpg,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NV
18017,ISIC_0029212.jpg,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,MEL
9489,ISIC_0033795.jpg,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NV
33128,ISIC_0069406.jpg,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,BKL
2496,ISIC_0026802.jpg,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,MEL
5040,ISIC_0029346.jpg,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NV
2438,ISIC_0026744.jpg,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,BKL
23615,ISIC_0054272.jpg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,AK
28699,ISIC_0062394.jpg,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NV


In [ ]:
path="/content/train"
shutil.rmtree(path)

TODO

Load Data  

Format data function  
Merge data function

Get Weighted Loss




Copy Chapter 7


In [ ]:
dls = ImageDataLoaders.from_df(
    gt2,
    path="/content/train/ISIC2018_Task3_Training_Input",
    csv_fname="/content/train/ISIC2018_Task3_Training_Input/Training_GroundTruth_Merged.csv",
    label_col='dx',
    item_tfms = Resize(460),
    batch_tfms = aug_transforms(size=224, min_scale=0.75),
    bs=64
)


In [ ]:
def get_weights(labels):
    cols = len(labels.columns)-2 #assumes 1 column for image ids
    weights = {}
    for i in range(cols+1):
        weights[i] = 1-np.mean(labels[labels.columns[i+1]].tolist())
    return weights

In [ ]:
#weights = get_weights(gt) #TODO: convert into pytorch tensor

model = xresnet50()
learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(), metrics=accuracy)
learn.fit_one_cycle(5, 3e-3)

epoch,train_loss,valid_loss,accuracy,time


KeyboardInterrupt: 

In [ ]:
# an example of normalization calculated on a batch of images
# (because we aren't using normalization yet, you'll see the mean and standard deviation are not very close to
# 0 and 1 respectively)
x, y = dls.one_batch()

x.mean(dim=[0,2,3]), x.std(dim=[0,2,3])

In [ ]:
def get_dls(batch_size, image_size):
  dls = ImageDataLoaders.from_df(
    gt2,
    path="/content/train/ISIC2018_Task3_Training_Input",
    csv_fname="/content/train/ISIC2018_Task3_Training_Input/Training_GroundTruth_Merged.csv",
    label_col='dx',
    item_tfms = Resize(460),
    batch_tfms = [*aug_transforms(size=image_size, min_scale=0.75), Normalize.from_stats(*imagenet_stats)],
    bs=batch_size
  )
  return dls

In [ ]:
dls = get_dls(64, 224)

# an example of normalization calculated on a batch of images
# (because we are using normalization now, the mean and standard deviation are very close to 0 and 1 respectively)
x, y = dls.one_batch()
print(x.mean(dim=[0,2,3]), x.std(dim=[0,2,3]))

# does this normalization improve our model? Let's see ...
model = xresnet50()
learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(), metrics=accuracy)
learn.fit_one_cycle(5, 3e-3)

## Progressive Resizing

In [ ]:
dls = get_dls(128,128)
learn = Learner(dls, xresnet50(), loss_func=CrossEntropyLossFlat(), metrics=accuracy)
learn.fit_one_cycle(4, 3e-3)

# simply replace the `Learner.dls` with new `DataLoaders` and continue traning.
learn.dls = get_dls(64, 224)
learn.fine_tune(5, 1e-3)

## TTA

In [ ]:
# you can pass any `DataLoaders` to `tta()` (by default it uses your validation `DataLoader`)
preds, targs = learn.tta()
accuracy(preds, targs).item()

## MixUp

In [ ]:
model = xresnet50()
learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(), metrics=accuracy, cbs=MixUp)
learn.fit_one_cycle(5, 3e-3)

## Label Smoothing

In [ ]:
model = xresnet50()
learn = Learner(dls, model, loss_func=LabelSmoothingCrossEntropy(), metrics=accuracy)
learn.fit_one_cycle(5, 3e-3)